In [1]:
import pandas as pd
import numpy as np
from nltk.tokenize import sent_tokenize
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix
from sklearn.preprocessing import LabelEncoder

from sentence_transformers import SentenceTransformer
from sklearn.svm import SVC

# Approach Explanation

Sentence-BERT (SBERT), a modification of the pretrained BERT network that use siamese and triplet network structures to derive semantically meaningful sentence embeddings that can be compared using cosine-similarity.

1. Documents are tokenized into sentences.
2. Sentence Embeddings is extracted from the Sentence transformer : paraphrase-MiniLM-L6-v2 model for each sentence in documents.<br>
    <b>Note:</b> Embdedding size is 384 for each sentence.
3. Sentence embeddings are averaged out to create document emedding
4. document embeddings along with the label is used for training SVM Model.

In [2]:
train_data = pd.read_csv('train_dataset_multiclass.csv',index_col=0)
print(train_data.shape,end="\n\n")
print(train_data['Issue'].value_counts())
train_data.head()

(5908, 2)

Problem with a purchase shown on your statement    1993
Incorrect information on your report               1191
Managing an account                                 935
Fees or interest                                    904
Other features, terms, or problems                  885
Name: Issue, dtype: int64


,Consumer complaint narrative,Issue
57519,The Citibank Credit Card company is a ridiculo...,Problem with a purchase shown on your statement
2790,Hi I went to a local bar on the evening of XX/...,Problem with a purchase shown on your statement
54277,I transferred XXXX ThankYou Points ( Citibank ...,"Other features, terms, or problems"
587,I was in XXXX XXXX and used my CITIBANK credit...,Problem with a purchase shown on your statement
12420,I clicked a promotion on citibank 's website t...,Managing an account


In [3]:
test_data = pd.read_csv('test_dataset_multiclass.csv',index_col=0)
print(test_data.shape,end="\n\n")
print(test_data['Issue'].value_counts())
test_data.head()

(2532, 2)

Problem with a purchase shown on your statement    854
Incorrect information on your report               510
Managing an account                                401
Fees or interest                                   388
Other features, terms, or problems                 379
Name: Issue, dtype: int64


,Consumer complaint narrative,Issue
17357,"Back in 2015, we noticed a charge on our Macy ...",Fees or interest
6524,"- On XX/XX/XXXX, I purchased 1 ticket XXXX {$3...",Problem with a purchase shown on your statement
23689,I hope all is well ... \nThe reason why Im wri...,Fees or interest
6960,Citibank had requested some XXXX documents ear...,Managing an account
7204,On XX/XX/2021 I authorized a payment of amount...,Managing an account


In [4]:
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [5]:
train_data['Embeddings'] = train_data['Consumer complaint narrative'].apply(lambda x: np.mean(model.encode(sent_tokenize(x)),axis=0))

In [6]:
train_data['Label'] = (train_data['Issue']=='Problem with a purchase shown on your statement').astype(int)

In [7]:
train_data['Label'].value_counts()

0    3915
1    1993
Name: Label, dtype: int64

In [8]:
test_data['Embeddings'] = test_data['Consumer complaint narrative'].apply(lambda x: np.mean(model.encode(sent_tokenize(x)),axis=0))

In [9]:
test_data['Label'] = (test_data['Issue']=='Problem with a purchase shown on your statement').astype(int)

In [10]:
test_data['Label'].value_counts()

0    1678
1     854
Name: Label, dtype: int64

In [11]:
X_train = train_data['Embeddings'].apply(pd.Series)
y_train = train_data['Label']

X_test = test_data['Embeddings'].apply(pd.Series)
y_test = test_data['Label']

In [12]:
clf = SVC()
clf.fit(X_train,y_train)

SVC()

In [13]:
predictions = clf.predict(X_test)

In [14]:
accuracy_score(y_test,predictions)

0.8854660347551343

In [15]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.89      0.94      0.92      1678
           1       0.87      0.78      0.82       854

    accuracy                           0.89      2532
   macro avg       0.88      0.86      0.87      2532
weighted avg       0.88      0.89      0.88      2532



In [16]:
print(confusion_matrix(y_test,predictions))

[[1579   99]
 [ 191  663]]


In [17]:
TN,FP,FN,TP = confusion_matrix(y_test,predictions).flatten()

print(TN,FP,FN,TP)

1579 99 191 663


In [18]:
FPR = FP/(TN+FP)

FPR

0.05899880810488677